1- Seleccionar un target del data set

2- Que problema puedo solucionar con esa columna ? (regresion o clasificacions )

3- Comenzar con la preparacion del data set (x,y) para lebels y features

4-Eliminar datos vacios o rellenar

5- Dummy hotencoding (transform data to numeric)

6- x todas las features y la y son las lebels que se toman de la columna que escogi

In [2]:
import pandas as pd
# Ruta al archivo local
file_path = '/content/Indicadores_municipales_sabana_DA.csv'

# Cargar el conjunto de datos en un DataFrame de pandas
df = pd.read_csv(file_path, encoding='latin-1')

# Mostrar las primeras filas del DataFrame para verificar que se haya cargado correctamente
df.head()

,ent,nom_ent,mun,clave_mun,nom_mun,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,1,Aguascalientes,1,1001,Aguascalientes,794304,30.531104,2.264478,28.266627,27.983320,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,1,Aguascalientes,2,1002,Asientos,48592,67.111172,8.040704,59.070468,22.439389,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,1,Aguascalientes,3,1003,Calvillo,53104,61.360527,7.241238,54.119289,29.428583,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,1,Aguascalientes,4,1004,Cosío,14101,52.800458,4.769001,48.031458,27.128568,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,1,Aguascalientes,5,1005,Jesús María,101379,45.338512,6.084037,39.254475,26.262912,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083


In [3]:
columns_to_exclude = ["ent", "nom_ent", "mun","nom_mun", "clave_mun","N_ic_rezedu"]
X = df.drop(columns_to_exclude, axis=1)
N_mun = df['nom_mun']
y = df["N_ic_rezedu"]

print("data already divided")



data already divided


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

label_encoder = LabelEncoder()
X['gdo_rezsoc00'] = label_encoder.fit_transform(X['gdo_rezsoc00'])
X['gdo_rezsoc05'] = label_encoder.fit_transform(X['gdo_rezsoc05'])
X['gdo_rezsoc10'] = label_encoder.fit_transform(X['gdo_rezsoc10'])

imputer = SimpleImputer(strategy="mean")

X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X_imputed)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

In [6]:
import numpy as np
def calcular_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [7]:
def descenso_gradiente(X, y, X_test, y_test, learning_rate=0.001, epochs=10000):
    num_muestras, num_caracteristicas = X.shape
    pesos = np.zeros(num_caracteristicas)  # Inicializar pesos a cero
    sesgo = 0  # Inicializar sesgo a cero

    errores_entrenamiento = []  # Almacenar errores durante el entrenamiento
    errores_prueba = []  # Almacenar errores en el conjunto de prueba

    for _ in range(epochs):
        # Calcular predicciones
        predicciones = np.dot(X, pesos) + sesgo
        predicciones_test = np.dot(X_test, pesos) + sesgo

        # Calcular el error en el conjunto de entrenamiento y prueba
        error_entrenamiento = calcular_error(y, predicciones)
        error_prueba = calcular_error(y_test, predicciones_test)

        # Almacenar errores
        errores_entrenamiento.append(error_entrenamiento)
        errores_prueba.append(error_prueba)

        # Calcular gradientes
        gradientes_pesos = -(2/num_muestras) * np.dot(X.T, (y - predicciones))
        gradiente_sesgo = -(2/num_muestras) * np.sum(y - predicciones)

        # Actualizar pesos y sesgo usando el descenso del gradiente
        pesos -= learning_rate * gradientes_pesos
        sesgo -= learning_rate * gradiente_sesgo
         # Impresiones intermedias

    return pesos, sesgo, errores_entrenamiento, errores_prueba

In [8]:
pesos, sesgo, errores_entrenamiento, errores_prueba = descenso_gradiente(X_train, Y_train, X_test, Y_test)
y_pred = np.dot(X_test, pesos) + sesgo

mse = calcular_error(Y_test, y_pred)

print(f"Error Cuadrático Medio: {mse}")

Error Cuadrático Medio: 12559199.935958525


In [9]:
import random

indice_aleatorio = random.choice(range(len(Y_test)))

nombre_municipio = N_mun.iloc[indice_aleatorio]
prediccion_desigualdad = y_pred[indice_aleatorio]
desigualdad_actual = Y_test.iloc[indice_aleatorio]

# Imprimir la información del municipio seleccionado al azar
print(f"Para el municipio {nombre_municipio}, se predice que el numero de personas con rezago social es {prediccion_desigualdad}. El valor real de rezago es {desigualdad_actual}")

Para el municipio San Agustín Metzquititlán, se predice que el numero de personas con rezago social es 7238.015533473906. El valor real de rezago es 7467
